In [410]:
import numpy as np
from numpy.random import uniform, normal, multivariate_normal, exponential
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt

In [475]:
#it's not quite this. 3.2 does some sort of learning. how?
#we're measuring the likelihood (via the pdf) of x given our parameters and the result.
#THEN we combine that with our prior.
#Then we have a posterior belief about how possible these parameters are given the data we just saw and our prior belief.
#now we binarize this belief stochastically! and move onto the next iteration.

In [639]:
#metropolis-hastings functions
def sample(data: np.ndarray, N: int, B: int, start_theta: tuple, search_breadth: float=1, lag=10):
    """Takes N samples via the Metropolis-Hastings algorithm, with B burn-in samples."""
    theta = start_theta
    for b in range(B): #burnin samples
        results = step(data, theta, search_breadth)
        theta = results['theta']
    
    samples = []
    for n in range(N): #real samples
        results = step(data, theta, search_breadth)
        theta = results['theta']
        samples.append(results)
        for _ in range(lag): #thrown away samples to reduce autocorrelation
            results = step(data, theta, search_breadth)
            theta = results['theta']
    return samples

def step(data: np.ndarray, theta: tuple, search_breadth: float):
    """Takes one step in the Metropolis-Hastings algorithm by generating a new theta and comparing to a given theta."""
    theta_prime = sample_theta(theta, search_breadth) #sample a new set of parameters
    acceptance_probability = min(1,calc_acceptance_prob(theta, theta_prime, data, search_breadth)) #calculate the probability of acceptance
    accepted = acceptance_probability >= uniform() #probabilistically determine acceptance 
    return {'accepted': accepted, 'acceptance_prob': acceptance_probability, 'theta': theta_prime if accepted else theta} #return results, update theta if samples accepted

def sample_theta(theta: tuple, search_breadth: float=1):
    """Samples theta parameters--slope, intercept, and standard deviation."""
    a,b,sigma = theta
    a,b = multivariate_normal([a,b], [[search_breadth**2,0],[0,search_breadth**2]])
    sigma = exponential(sigma*search_breadth)
    theta = a,b,sigma
    return theta

def calc_acceptance_prob(theta: tuple, theta_prime: tuple, data: np.ndarray, search_breadth: float):
    """Calculates acceptance probability by using a Bayesian linear model."""
    theta_likelihood = likelihood(theta, data)
    theta_prior = prior(theta)
    
    theta_p_likelihood = likelihood(theta_prime, data)
    theta_p_prior = prior(theta)
    
    acceptance_ratio = (theta_p_likelihood/theta_likelihood) * ((theta_p_prior/theta_prior) * proposal_ratio(theta, theta_prime))
    return acceptance_ratio

#bayesian functions
def likelihood(theta: tuple, data: np.ndarray):
    """Calculates the likelihood component of our linear model by measuring our parameters theta on the given data."""
    a,b,sigma = theta
    x,y = data[0],data[1]
    likelihoods = sc.stats.norm.pdf(y, loc=a*x+b, scale=sigma) #find the likelihood of a sample given a normal distribution specified by our parameters and the data
    return np.prod(likelihoods)

def prior(theta: tuple):
    """Calculates the prior component of our linear model, specified """
    a,b,sigma = theta
    a_prob, b_prob = sc.stats.multivariate_normal.pdf([a,b]) #cov defaults to 1
    sigma_prob = sc.stats.expon.pdf(sigma)
    return np.prod([a_prob,b_prob,sigma_prob])

def proposal_ratio(theta: tuple, theta_prime: tuple):
    """Offsets bidirectionality of chained samples. Should only ever evaluate to 1 because we're using the normal 
    and exponential distributions, but included so I can call this a Metropolis-HASTINGS algorithm."""
    old_given_new = sc.stats.multivariate_normal.pdf([a,b],[a_p,b_p])*sc.stats.expon.pdf(sigma)
    new_given_old = sc.stats.multivariate_normal.pdf([a_p,b_p],[a,b])*sc.stats.expon.pdf(sigma_p)
    return old_given_new/new_given_old

In [629]:
step(data, (0,0,1), 0.5)

{'accepted': False,
 'acceptance_prob': 1.465133273728559e-31,
 'theta': (0, 0, 1)}

In [ ]:
#data = np.zeros((2,100))
data = np.random.rand(2,100) #meaningless data. all zeros
samples = sample(data, 1000, 5000, (0,0,1), 1, 10)
df = pd.DataFrame(samples)
df[['a','b','sigma']] = pd.DataFrame(df['theta'].tolist(), index=df.index)

In [ ]:
data

In [ ]:
df

In [ ]:
plt.plot(range(len(df)),df['a'], label='slope')
plt.plot(range(len(df)),df['b'], label='intercept')
plt.plot(range(len(df)),df['sigma'], label='variance')
plt.plot(range(len(df)),df['acceptance_prob'], label='accept_prob')
plt.legend()

Questions:
- In the zero case, the stationary distribution should be the thetas right? The likelihood component reflects the thetas, and the prior reflects the thetas + some noise. So why does it converge to some number and then drop to a very low acceptance probability?

Great! Basic example of MH working. Now need to implement the Bayesian formulation. in calc_acceptance_prob().

### Data

In [168]:
#okay, let's pretend this is our NFL data. we have 5 predictor variables for our linear model. then we have our response.
#each of the first 5 are normal variables, with their own mean and variance. 
data = multivariate_normal(np.array([5,0,-2,1,3,6]),np.ones((6,6)),100)
data[:5]

array([[ 6.27823707,  1.27823707, -0.72176293,  2.27823707,  4.27823707,
         7.27823707],
       [ 3.49665799, -1.50334201, -3.50334201, -0.50334201,  1.49665799,
         4.49665799],
       [ 4.92325527, -0.0767447 , -2.0767447 ,  0.9232553 ,  2.9232553 ,
         5.9232553 ],
       [ 5.2091501 ,  0.20915008, -1.79084992,  1.20915008,  3.20915008,
         6.20915008],
       [ 5.65139933,  0.65139933, -1.34860067,  1.65139933,  3.65139933,
         6.65139933]])

In [181]:
A = np.array([2,2,2,2,2,2])
B = np.array([1,2,3,2,1,2])
pe = A*data+B
#y = multivariate_normal(A*data+B)
pe

array([[ 1.35564741e+01,  4.55647413e+00,  1.55647413e+00,
         6.55647413e+00,  9.55647413e+00,  1.65564741e+01],
       [ 7.99331598e+00, -1.00668402e+00, -4.00668402e+00,
         9.93315976e-01,  3.99331598e+00,  1.09933160e+01],
       [ 1.08465105e+01,  1.84651060e+00, -1.15348940e+00,
         3.84651060e+00,  6.84651060e+00,  1.38465106e+01],
       [ 1.14183002e+01,  2.41830016e+00, -5.81699836e-01,
         4.41830016e+00,  7.41830016e+00,  1.44183002e+01],
       [ 1.23027987e+01,  3.30279866e+00,  3.02798656e-01,
         5.30279866e+00,  8.30279866e+00,  1.53027987e+01],
       [ 1.39996770e+01,  4.99967704e+00,  1.99967704e+00,
         6.99967704e+00,  9.99967704e+00,  1.69996770e+01],
       [ 8.09741629e+00, -9.02583705e-01, -3.90258371e+00,
         1.09741629e+00,  4.09741629e+00,  1.10974163e+01],
       [ 1.16623132e+01,  2.66231323e+00, -3.37686774e-01,
         4.66231323e+00,  7.66231323e+00,  1.46623132e+01],
       [ 9.91265182e+00,  9.12651780e-01, -2.087

In [ ]:
#we want to estimate a and b, which are the betas and the intercept for the above values. then we also want to estimate sigma
#which is the variance of the data from the estimates.

#so: normal(a*data+b,sigma) is a perfect model of the data.

In [ ]:
#the data is 6x100. so we should learn an (a,b) or 5x1 and 1x1 respectively. and sigma is 5x1?

things to define:
- what our parameters are and their shape
    - a,b,sigma
    - 
- how to sample them
    - we want to sample a and b from a multivariate normal distribution centered at the previous iteration step.
        - this may be the redherring for mvn

is this tutorial doing multiD or just oneD?
- clearly just 1D in 3.0, but maybe manyD for 3.*



In [291]:
def sample_theta(theta: tuple, search_breadth: float=1):
    a,b,sigma = theta
    a,b = multivariate_normal([a,b], [[search_breadth**2,0],[0,search_breadth**2]])
    sigma = exponential(sigma*search_breadth)
    theta = a,b,sigma
    return theta

sample_theta((0,0,0.5),1)

(-0.971266488090688, -1.2501228151976096, 0.3146455037657163)

In [222]:
x,y = normal(0,1,100),np.zeros(100)
sc.norm.pdf(y, loc=theta[0]*x+theta[1], scale=theta[2])

array([6.28517630e-07, 8.61612527e-07, 3.72499935e-05, 3.06261765e+00,
       2.05475660e+00, 1.19289869e+00, 9.63188685e-07, 2.51022956e-33,
       1.16010310e-18, 1.31545566e+00, 1.82300506e-02, 9.53778179e-05,
       6.91375324e-07, 1.79787728e-01, 5.60159653e-26, 7.33561524e-08,
       4.68767850e-44, 1.07561276e-08, 5.70463855e-22, 5.93423003e-20,
       1.15704608e-04, 1.03830085e-05, 5.88124143e-02, 4.09619496e-38,
       3.71656581e-09, 1.57663653e+00, 7.87361608e-31, 2.05049795e-11,
       2.08686538e-15, 1.62987048e+00, 1.16016816e+00, 1.02271655e-07,
       2.37664364e+00, 5.27739327e-43, 2.42468039e-25, 1.33446540e-10,
       9.31943929e-16, 7.64401022e-04, 1.50300516e-19, 2.52863198e-16,
       9.79212128e-18, 9.43016402e-07, 1.20091188e-17, 3.92926163e-25,
       2.68798928e-29, 6.69465625e-05, 2.83211561e+00, 1.81445978e-02,
       6.89346020e-09, 2.34147711e-11, 8.81959094e-19, 1.02186236e-01,
       1.45480160e+00, 4.32465313e-01, 2.21699631e+00, 2.59671419e+00,
      

In [230]:
sc.norm.pdf(0, loc=7, scale=1)

9.134720408364595e-12

In [217]:
normal(theta[0]*data+theta[1],theta[2])


array([-5.02868025e-01, -4.16294507e-01,  7.09958245e-01,  1.22012581e+00,
       -1.19914619e+00, -7.69109937e-01, -9.82422332e-01,  7.62666093e-01,
       -1.75690033e-01,  3.13293572e-01,  6.42175030e-01,  1.45250718e-01,
        1.06081801e+00, -4.15422208e-02,  9.28536463e-02, -1.74487250e-01,
        7.09712560e-01, -3.54803311e-01, -4.00141200e-02, -1.49299707e+00,
        9.97838280e-01, -2.21008174e-01, -9.28410819e-01,  3.54232791e-01,
        3.14851811e-01,  5.21903810e-01,  6.02511645e-01, -7.15544926e-01,
       -3.20368932e-01, -8.35556016e-01,  2.09635426e-01, -3.79137533e-01,
        3.88775810e-01, -4.44222358e-01, -9.21918995e-01,  1.02948500e+00,
       -9.06421053e-04,  2.00415476e-03,  1.05449000e-01,  1.22368755e+00,
        1.05238721e-01, -6.21555775e-01,  1.51926326e+00, -1.84168971e-01,
       -9.46214882e-01, -9.59552065e-02,  6.88855164e-01,  8.15364127e-02,
       -1.41517163e-01,  1.38789413e+00, -7.16155884e-01,  1.16608836e+00,
        5.83377042e-01,  